In [4]:
"""Module for line segment transformation."""
import math
#from src.GCodeContainer import GcodeContainer

(1) check relative position of segment:
- where is inside / outside?
- get center point (x1 = x0 + dx/2), length, slope of segment

(2) get transform parameters
- get outer normal func of segment
- get t_func value, t_func slope at center point

(3) do transformation
- normal shift center point by t_func value
- get new slope at shifted center point

(4) couple new t_segment with last t_segment
- get new line func from new slope & new center point
- get new line start point: f0 = f1 at x0

### Mathe...
Addition der Winkel --> Anstieg der resultierenden Geraden errechnen:

(1) Ausgangsgerade: Anstieg m = dx / dy  

(2) Transformationsfunktion: lokalen Anstieg n approximieren über 2 benachbarte Funktionswerte  

(3) Winkel (mit dem Koordinatenursprung) beider Anstiege addieren:  
arctan(m) + arctan(n) = arctan((m + n) / (1 - m * n)) für m*n < 1  

resultierender Anstieg:  
(4) tan(arctan((m + n) / (1 - m * n))) für m*n < 1  

(5) tan( Pi + arctan((m + n) / (1 - m * n))) für m*n > 1 und m > 0  

(6) tan( -Pi + arctan((m + n) / (1 - m * n))) für m*n > 1 und m < 0

In [20]:
# Transformationsfunktion
def f_trans(arg_x: float, scale: float=1.0) -> float:
    return (math.sin(arg_x) * scale)

In [28]:
# Berechnung für einen einzelnen Punkt B des Dreiecks ABC:
Arg_f = 0.0

# Punkte A, B, C
A = (2.0, 3.0)
B = (5.0, 5.0)
C = (8.0, 4.0)

# Anstieg m der Strecke AB; Anstieg n ihrer Orthogonale
dx_AC = C[0] - A[0]
dy_AC = C[1] - A[1]
m = dy_AC / dx_AC
n = -1 / m

# Fußpunkt der Orthogonalen durch B auf AC
xh = (B[1] - A[1]) / (m + (1 / m))
yh = A[1] + (n * xh)
Bh = (xh, yh)

# Fortschreibung des Arguments der Transformationsfunktion um dArg_f
dx_ABh = Bh[0] - A[0]
dy_ABh = Bh[1] - A[1]
len_ABh = math.sqrt(dx_ABh**2 + dy_ABh**2)
Arg_f += len_ABh

# neuer Punkt B
fb = f_trans(Arg_f, scale=3.0)
print(f"Argument der Transformationsfunktion: {Arg_f:.4f}; Funktionswert {fb:.4f}")
Bi = (
    (B[0] - math.sqrt(fb**2 / (1 + n**2))),
    (B[1] - (n * math.sqrt(fb**2 / (1 + n**2))))
)

l_AC = math.sqrt(dx_AC**2 + dy_AC**2)
print(f"Länge AC: {l_AC:.4f}, Anstieg m {m:.4f}, Anstieg n {n:.4f}")
print(f"Neuer Punkt Bi: {Bi}")

# Längenänderung AB - ABi: mehr / weniger Materialaustrag?
dx_AB = B[0] - A[0]
dy_AB = B[1] - A[1]
len_AB = math.sqrt(dx_AB**2 + dy_AB**2)
dx_ABi = Bi[0] - A[0]
dy_ABi = Bi[1] - A[1]
len_ABi = math.sqrt(dx_ABi**2 + dy_ABi**2)
d_len = (len_ABi / len_AB) - 1
print(f"Länge AB: {len_AB}, Länge ABi: {len_ABi}, d_len: {(d_len * 100):.2f}%.")

Argument der Transformationsfunktion: 2.5680; Funktionswert 1.6280
Länge AC: 6.0828, Anstieg m 0.1667, Anstieg n -6.0000
Neuer Punkt Bi: (4.732362709587209, 6.605823742476747)
Länge AB: 3.605551275463989, Länge ABi: 4.5241320536155625, d_len: 25.48%.


In [ ]:
HORIZ = False
VERT = True
LTR = False
RTL = True

class LineSegTransformer:
    def __init__(self, point_0, point_1) -> None:
        """initialize class instance

        _extended_summary_

        Args:
            point_0 (tuple of floats x, y): last point coords
            point_1 (tuple of floats x, y): current point coords
            line_0_props (_type_): line equation properties of last segment,
                                    None if unknown
        """
        self.p0 = point_0
        self.p1 = point_1

        self.dx = self.p1["x"] - self.p0["x"]
        self.dy = self.p1["y"] - self.p0["y"]
        self.center = {
            "x": self.p0["x"] + (self.dx / 2),
            "y": self.p0["y"] + (self.dy / 2)
            }

        # some things we don't know yet
        self.p0_tf = None
        self.line0 = None
        self.center_tf = None

        # todo efficiency: use trigo or sqrt?
        # length of current line segment before tf
        self.length = math.sqrt(self.dx**2 + self.dy**2)

        abs_dx = abs(self.dx)
        abs_dy = abs(self.dy)
        self.orientation = abs_dx < abs_dy
        if self.orientation == VERT:
            self.direction = self.dy / abs_dy
        else:
            self.direction = self.dx / abs_dx


    def fit(self, arg_0: float, trans_func) -> None:
        transform_arg = arg_0 + (self.length / 2)
        # length of orthogonal transformation vector, sign -> direction
        self.tf_ortho_length = trans_func(transform_arg)
        
        # approximate slope of transform function at tf point
        # by minimal differential calculus
        diff = self.length / 5_000
        tf_val_0 = trans_func(transform_arg - diff)
        tf_val_1 = trans_func(transform_arg + diff)
        self.tf_func_center_slope = (tf_val_1 - tf_val_0) / (diff * 2)

        return None

    def transform(self, line0_params) -> None:
        pass

        return None